In [ ]:
# import sys
# 
# from tests.smoke_tests.olesik_et_al_2022.test_simulation import settings
# 
# if 'google.colab' in sys.modules:
#     !pip --quiet install open-atmos-jupyter-utils
#     from open_atmos_jupyter_utils import pip_install_on_colab
#     pip_install_on_colab('PyMPDATA-examples')

In [1]:
import matplotlib.pyplot as plt

import os

import pandas

# os.environ["NUMBA_DISABLE_JIT"] = "1"

from asian_option import AsianArithmetic, Settings
import numpy as np
from matplotlib import pyplot
import pandas as pd
from ipywidgets import IntProgress
# from open_atmos_jupyter_utils import show_plot, show_anim
from PyMPDATA_examples.Magnuszewski_et_al_2025.common import OPTIONS
from monte_carlo import BSModel, FixedStrikeGeometricAsianOption, FixedStrikeArithmeticAsianOption
import time
from tqdm import tqdm
import PyMPDATA_examples.utils.financial_formulae.asian_option as asian_analytic
from PyMPDATA_examples.Magnuszewski_et_al_2025 import barraquand_data

pandas.options.display.float_format = '{:,.3f}'.format

In [2]:
s_min = 50
s_max = 200
mc_n_paths = [1000, 10000]
mc_seed = 42
mc_path_points = 1000
spot = 100
risk_free_rate=0.1


In [3]:
def run_numeric_and_mc(params, *, nx, ny, nt, variant):
    settings = Settings(**params, r=risk_free_rate, S_max=s_max, S_min=s_min)
    mc_model = BSModel(T=params['T'],
                       sigma=params['sgma'],
                       r=risk_free_rate,
                       M=mc_path_points,
                       S0=spot,
                       seed=mc_seed)
    simulations = {k:AsianArithmetic(settings, nx=nx, ny=ny, nt=nt, variant=variant, options=OPTIONS[k]) for k in OPTIONS}
    results = {}
    
    for k, simulation in simulations.items():
        simulation.step(simulation.nt)
        simulation_price = simulation.solver.advectee.get()[:, 0]
        results[k] = np.interp(spot, simulation.S, simulation_price)
    for mc_n_path in mc_n_paths:
        arithmetic_option = FixedStrikeArithmeticAsianOption(params['T'], params['K'], variant, mc_model, mc_n_path)
        results[f"MC_{mc_n_path}"] = arithmetic_option.price_by_mc()
    return results

In [4]:
discretization_parameters = {
    (.1,.25,95):{'nx': 201, 'ny': 400, 'nt': 1500},
    (.1,.25,100):{'nx': 201, 'ny': 400, 'nt': 1500},
    (.1,.25,105):{'nx': 201, 'ny': 400, 'nt': 1500},
    (.1,.5,95):{'nx': 201, 'ny': 300, 'nt': 1500},
    (.1,.5,100):{'nx': 201, 'ny': 300, 'nt': 1500},
    (.1,.5,105):{'nx': 201, 'ny': 300, 'nt': 1500},
    (.1,1,95):{'nx': 251, 'ny': 250, 'nt': 1500},
    (.1,1,100):{'nx': 251, 'ny': 250, 'nt': 1500},
    (.1,1,105):{'nx': 251, 'ny': 250, 'nt': 1500},
    (.2,.25,95):{'nx': 251, 'ny': 250, 'nt': 1000},
    (.2,.25,100):{'nx': 251, 'ny': 250, 'nt': 1000},
    (.2,.25,105):{'nx': 251, 'ny': 250, 'nt': 1000},
    (.2,.5,95):{'nx': 101, 'ny': 200, 'nt': 1000},
    (.2,.5,100):{'nx': 101, 'ny': 200, 'nt': 1000},
    (.2,.5,105):{'nx': 101, 'ny': 200, 'nt': 1000},
    (.2,1,95):{'nx': 101, 'ny': 200, 'nt': 1500},
    (.2,1,100):{'nx': 101, 'ny': 200, 'nt': 1500},
    (.2,1,105):{'nx': 101, 'ny': 200, 'nt': 1200},
    (.4,.25,95):{'nx': 101, 'ny': 110, 'nt': 2200},
    (.4,.25,100):{'nx': 101, 'ny': 200, 'nt': 1000},
    (.4,.25,105):{'nx': 101, 'ny': 200, 'nt': 1500},
    (.4,.5,95):{'nx': 101, 'ny': 110, 'nt': 1000},
    (.4,.5,100):{'nx': 101, 'ny': 110, 'nt': 1000},
    (.4,.5,105):{'nx': 101, 'ny': 110, 'nt': 1000},
    (.4,1,95):{'nx': 101, 'ny': 100, 'nt': 1800},
    (.4,1,100):{'nx': 101, 'ny': 100, 'nt': 1780},
    (.4,1,105):{'nx': 101, 'ny': 100, 'nt': 1760},
}

In [6]:
barraquand_df = pd.DataFrame(columns=barraquand_data.headers)
for line in barraquand_data.table.strip('\n').split('\n'):
    data_row = line.split(',')
    if len(data_row) > 0:
        barraquand_df.loc[len(barraquand_df)] = data_row
barraquand_df['call_price'] = barraquand_df['call_price'].astype(float)
barraquand_df['put_price'] = barraquand_df['put_price'].astype(float)

In [7]:
def calculate_row(row_idx):
    row_data = barraquand_df.iloc[row_idx].astype(float)
    nx,ny,nt = discretization_parameters[(row_data['sigma'], row_data['T'], row_data['K'])].values()
    simulation_params = {
        'sgma':row_data['sigma'],
        'T':row_data['T'],
        'K':row_data['K']
    }
    call_price = row_data['call_price']
    put_price = row_data['put_price']
    results_call = run_numeric_and_mc(simulation_params, nx=nx, ny=ny, nt=nt, variant='call')
    results_put = run_numeric_and_mc(simulation_params, nx=nx, ny=ny, nt=nt, variant='put')
    return {k: round(v,3) for k, v in results_call.items()}, {k: round(v,3) for k, v in results_put.items()}, simulation_params, call_price, put_price

In [10]:
results_header = ['sigma', 'T', 'K', 'BP_call', 'UPWIND_call', 'MPDATA_call', 'MC_1000_call', 'MC_10000_call', 'BP_put', 'UPWIND_put', 'MPDATA_put', 'MC_1000_put', 'MC_10000_put']
results_df = pd.DataFrame(columns=results_header)
for i in tqdm(range(27)):
    call, put, params, call_bp, put_bp = calculate_row(i)
    new_row = [*params.values(), call_bp, *call.values(), put_bp, *put.values()]
    # print(new_row)
    results_df.loc[len(results_df)] = new_row
results_df['K'] = results_df['K'].astype(int)

100%|██████████| 27/27 [03:19<00:00,  7.38s/it]


In [11]:
display(results_df)

,sigma,T,K,BP_call,UPWIND_call,MPDATA_call,MC_1000_call,MC_10000_call,BP_put,UPWIND_put,MPDATA_put,MC_1000_put,MC_10000_put
0,0.100,0.250,95,6.132,6.861,6.051,6.031,6.116,0.013,0.737,0.012,0.022,0.010
1,0.100,0.250,100,1.869,3.496,1.853,1.835,1.852,0.626,2.244,0.626,0.702,0.623
2,0.100,0.250,105,0.151,1.403,0.238,0.142,0.148,3.785,5.032,3.794,3.886,3.796
3,0.100,0.500,95,7.248,8.109,7.142,7.106,7.215,0.046,0.904,0.041,0.067,0.041
4,0.100,0.500,100,3.100,4.750,3.046,3.026,3.071,0.655,2.298,0.650,0.743,0.653
5,0.100,0.500,105,0.727,2.395,0.820,0.710,0.718,3.039,4.700,3.058,3.183,3.056
6,0.100,1.000,95,9.313,10.080,9.240,9.138,9.280,0.084,0.837,0.077,0.120,0.078
7,0.100,1.000,100,5.279,6.661,5.222,5.170,5.253,0.577,1.938,0.568,0.676,0.576
8,0.100,1.000,105,2.313,3.999,2.346,2.299,2.294,2.137,3.798,2.139,2.330,2.140
9,0.200,0.250,95,6.500,7.742,6.418,6.369,6.474,0.379,1.624,0.368,0.452,0.369
